In [2]:
!pip install -q yfinance pandas_datareader scikit-learn pandas numpy



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:

import pandas as pd
import numpy as np
import json
import yfinance as yf

from sklearn.linear_model import ElasticNetCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit

try:
    from pandas_datareader import data as pdr
except ImportError:
    pdr = None

# -----------------------------
# 0) 설정
# -----------------------------
START = "2010-01-01"
END   = None  # None이면 오늘까지

# ✅ 너희 프로젝트 ETF로 바꾸고 싶으면 여기만 수정
TICKERS = {
    "nasdaq": "QQQ",  # 나스닥100 ETF
    "gold":   "GLD",  # 금 ETF
    "oil":    "CL=F", # WTI 선물(유가 팩터)
}

# FRED 코드 (가능하면 사용, 안 되면 자동 fallback)
FRED_SERIES = {
    "10y": "DGS10",   # 미국 10Y 금리
    "2y":  "DGS2",    # 미국 2Y 금리
    "vix": "VIXCLS",  # VIX 지수
}

FORECAST_HORIZON_MONTHS = 12  # 1년 기대수익률로 변환

# -----------------------------
# 1) 데이터 로드 헬퍼
# -----------------------------
def load_yf_price(ticker):
    # auto_adjust=False로 해서 원래 컬럼들 받기
    data = yf.download(ticker, start=START, end=END, progress=False, auto_adjust=False)

    # 우선 Adj Close 우선 사용, 없으면 Close 사용
    col = "Adj Close" if "Adj Close" in data.columns else "Close"
    px = data[col]

    # px가 Series이면 그대로, DataFrame이면 해당 ticker 컬럼 or 첫 컬럼 선택
    import pandas as pd
    if isinstance(px, pd.DataFrame):
        if ticker in px.columns:
            px = px[ticker]          # 지정 티커 컬럼
        else:
            px = px.iloc[:, 0]       # 첫 컬럼 fallback

    # 이름을 티커로 설정 (Series.name)
    px = px.rename(ticker)
    return px
    
def load_fred(series_code):
    if pdr is None:
        raise RuntimeError("pandas_datareader 설치 필요")
    return pdr.DataReader(series_code, "fred", START, END)[series_code]

def monthly_return(price_series):
    return price_series.resample("M").last().pct_change()

# -----------------------------
# 2) 데이터 수집
# -----------------------------
nasdaq_px = load_yf_price(TICKERS["nasdaq"])
gold_px   = load_yf_price(TICKERS["gold"])
oil_px    = load_yf_price(TICKERS["oil"])

nasdaq_ret = monthly_return(nasdaq_px).rename("nasdaq_ret")
gold_ret   = monthly_return(gold_px).rename("gold_ret")
oil_ret    = monthly_return(oil_px).rename("oil_ret")

# 금리/VIX (FRED 시도 → 실패하면 VIX만 yfinance로)
ten_y = two_y = None

vix = None
if pdr is not None:
    try:
        ten_y = load_fred(FRED_SERIES["10y"]).resample("M").last().rename("ten_y")
        two_y = load_fred(FRED_SERIES["2y"]).resample("M").last().rename("two_y")
        vix   = load_fred(FRED_SERIES["vix"]).resample("M").last().rename("vix")
        print("✅ FRED 금리/VIX 로드 성공")
    except Exception as e:
        print("⚠️ FRED 로드 실패 → VIX는 yfinance로 대체. 금리는 제외.")
        vix_px = load_yf_price("^VIX")
        vix = vix_px.resample("M").last().rename("vix")
else:
    print("⚠️ pandas_datareader 없음 → VIX는 yfinance로 대체. 금리는 제외.")
    vix_px = load_yf_price("^VIX")
    vix = vix_px.resample("M").last().rename("vix")


# -----------------------------
# 3) 팩터(X) 생성
# -----------------------------
def make_features(ret_series, oil_ret, vix, ten_y=None, two_y=None):
    df = pd.concat([ret_series, oil_ret, vix], axis=1)

    # 모멘텀(1M,3M,12M)
    df["mom_1m"]  = df[ret_series.name].shift(1)
    df["mom_3m"]  = df[ret_series.name].rolling(3).mean().shift(1)
    df["mom_12m"] = df[ret_series.name].rolling(12).mean().shift(1)

    # VIX 변화율(1M)
    df["vix_chg_1m"] = df["vix"].pct_change().shift(1)

    # 유가 3M 모멘텀
    df["oil_mom_3m"] = df["oil_ret"].rolling(3).mean().shift(1)

    # 금리 레벨/스프레드 (있을 때만)
    if ten_y is not None and two_y is not None:
        df = pd.concat([df, ten_y, two_y], axis=1)
        df["term_spread"] = (df["ten_y"] - df["two_y"]).shift(1)
        df["rate_lvl"]    = df["ten_y"].shift(1)

    df = df.dropna()
    return df


# -----------------------------
# 4) Elastic Net 학습 & 1년 기대수익률 산출
# -----------------------------
def forecast_expected_return(feature_df, target_col):
    X = feature_df.drop(columns=[target_col])
    y = feature_df[target_col]

    tscv = TimeSeriesSplit(n_splits=5)

    model = Pipeline([
        ("scaler", StandardScaler()),
        ("enet", ElasticNetCV(
            l1_ratio=[.1, .3, .5, .7, .9, 1.0],
            alphas=np.logspace(-4, 1, 50),
            cv=tscv,
            max_iter=20000
        ))
    ])

    model.fit(X, y)

    latest_X = X.iloc[[-1]]
    next_1m_ret_hat = float(model.predict(latest_X)[0])

    exp_1y = (1 + next_1m_ret_hat)**FORECAST_HORIZON_MONTHS - 1
    return exp_1y, next_1m_ret_hat, model


nasdaq_feat = make_features(nasdaq_ret, oil_ret, vix, ten_y, two_y)
gold_feat   = make_features(gold_ret,   oil_ret, vix, ten_y, two_y)

nasdaq_1y, nasdaq_1m, _ = forecast_expected_return(nasdaq_feat, "nasdaq_ret")
gold_1y,   gold_1m,   _ = forecast_expected_return(gold_feat,   "gold_ret")

# ✅ 적금 기대수익률: MVP는 고정금리(너희가 정한 값)로
saving_1y = 0.03

expected_returns = {
    "saving": round(saving_1y, 4),
    "nasdaq": round(nasdaq_1y, 4),
    "gold":   round(gold_1y, 4),
    "meta": {
        "nasdaq_next1m_hat": round(nasdaq_1m, 4),
        "gold_next1m_hat":   round(gold_1m, 4),
        "asof": str(nasdaq_feat.index[-1].date())
    }
}

print("📌 expected_returns:", expected_returns)

with open("expected_returns.json", "w", encoding="utf-8") as f:
    json.dump(expected_returns, f, ensure_ascii=False, indent=2)

print("✅ saved expected_returns.json")

C:\Users\myunghyeseong\AppData\Local\Temp\ipykernel_21284\3590920060.py:67: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  return price_series.resample("M").last().pct_change()
C:\Users\myunghyeseong\AppData\Local\Temp\ipykernel_21284\3590920060.py:86: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  ten_y = load_fred(FRED_SERIES["10y"]).resample("M").last().rename("ten_y")
C:\Users\myunghyeseong\AppData\Local\Temp\ipykernel_21284\3590920060.py:87: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  two_y = load_fred(FRED_SERIES["2y"]).resample("M").last().rename("two_y")
C:\Users\myunghyeseong\AppData\Local\Temp\ipykernel_21284\3590920060.py:88: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  vix   = load_fred(FRED_SERIES["vix"]).resample("M").last().rename("vix")


✅ FRED 금리/VIX 로드 성공
📌 expected_returns: {'saving': 0.03, 'nasdaq': 0.1479, 'gold': 0.09, 'meta': {'nasdaq_next1m_hat': 0.0116, 'gold_next1m_hat': 0.0072, 'asof': '2025-11-30'}}
✅ saved expected_returns.json
